In [1]:
![ ! -e "$HOME/ollama-installed" ] && curl -fsSL https://ollama.com/install.sh | sh && touch "$HOME/ollama-installed"
!pip -qq install langchain-community
!pip -qq install gradio

import gradio as gr
import subprocess
from langchain_community.llms import Ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━

In [2]:
g_LLAMA_MODEL = "llama3.2:3b"

commands = [
    ["bash", "-c", "nohup ollama serve > /dev/null 2>&1 &"],
]
for c in commands:
    print(c)
    process = subprocess.Popen(c, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    print(stdout.decode())
    print(stderr.decode())

# avoid starting ollama immediately, since it will fail
!sleep 2

# load models
!ollama pull {g_LLAMA_MODEL}
!ollama show {g_LLAMA_MODEL}
output = get_ipython().getoutput(f'ollama show {g_LLAMA_MODEL}' + '| grep "context length" | awk \'{print $3}\'')
g_llm_context_length = int(output[0])

['bash', '-c', 'nohup ollama serve > /dev/null 2>&1 &']


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏    0 B/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   0% ▕▏ 2.1 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   1% ▕▏  26 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   3% ▕▏  58 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   4% ▕▏  73 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   5% ▕▏ 104 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   6% ▕▏ 128 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff...   7% ▕▏ 139 MB/2.0 GB                  pull

In [3]:
class Ollama_txt2sql:
    def __init__(self, model, max_model_context_length):
      self.model = model
      self.max_model_context_length = max_model_context_length
      self.ollama_client = Ollama(model=model, temperature=0.3)

    def txt2sql(self, question, schema):
      context = f"========= Begin of DB schema =========\n{schema}\n\n========= End of DB schema =========\n"
      prompt = f"Please review the DB schema very carefully, create SQL based on user's question, think step by step\n{question}\n"
      messages = [
              {"role": "system", "content": "You are an Database expert, very good at create SQL."},
              {"role": "assistant", "content": context},
              {"role": "user", "content": prompt},
              ]

      response = self.ollama_client.invoke(input=messages)

      return response

In [4]:
# Default schema/question for demo purposes
default_schema = """
CREATE TABLE Students (
    student_id INT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255) UNIQUE
);

CREATE TABLE Teachers (
    teacher_id INT PRIMARY KEY,
    name VARCHAR(255),
    specialty VARCHAR(255)
);

CREATE TABLE Classes (
    class_id INT PRIMARY KEY,
    title VARCHAR(255),
    teacher_id INT,
    FOREIGN KEY (teacher_id) REFERENCES Teachers(teacher_id)
);

CREATE TABLE ClassroomAssignments (
    assignment_id INT PRIMARY KEY,
    class_id INT,
    classroom VARCHAR(255),
    class_time TIME,
    FOREIGN KEY (class_id) REFERENCES Classes(class_id)
);

CREATE TABLE StudentClassAssociations (
    association_id INT PRIMARY KEY,
    student_id INT,
    class_id INT,
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (class_id) REFERENCES Classes(class_id)
);
"""
default_question = "How many teachers teach only one class?"

In [8]:
# Define txt2sql
def txt2sql(question, schema):
  ollama_txt2sql = Ollama_txt2sql(g_LLAMA_MODEL, g_llm_context_length)
  return ollama_txt2sql.txt2sql(question=question, schema=schema)



In [9]:
# Define gradio GUI
with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column():
      schema_input = gr.Textbox(label="DB Schema",
                                lines=30,
                                value=default_schema)
    with gr.Column():
      with gr.Row():
        question_input = gr.Textbox(label="Question",
                                       lines=3,
                                       value=default_question)
      with gr.Row():
        submit_button = gr.Button(value="TXT2SQL")
      with gr.Row():
        answer_output = gr.Textbox(lines=20, label="SQL")

    submit_button.click(fn=txt2sql,
                        inputs=[question_input, schema_input],
                        outputs=[answer_output])

In [10]:
# Launch gradio interface
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://93d64fa53b5b2d5a1b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
